In [1]:
import sys
sys.path.insert(0, "D:\\Documents\\food_recipe_gen\\recipe_1m_analysis")
from os import listdir
from os.path import isfile, join
import json
import pandas as pd 

from ingr_normalization import normalize_ingredient

In [2]:
PATH = "D:\\Google Drive\\Catherning Folder\\THU\\Thesis\\Recipe datasets\\Yummly28"
PATH2 = "D:\\Documents\\Recipe datasets\\metadata27638_yummly28"
FOLDER_PATH = join(PATH,"metadata27638")
raw_present = False

In [3]:
if not raw_present:
    files = [f for f in listdir(PATH2) if isfile(join(PATH2, f))]
    NB_RECIPE = len(files)
    recipes=[]
    for i in range(NB_RECIPE):
        with open(join(PATH2,files[i]),encoding="utf8") as json_file:
            dict_data = json.load(json_file)
        # TODO use nutrition estimages, totalTime (in s?) in the future, 
        # useless keys for now:
        # "totalTime","source","images","id","totalTimeInSeconds",'attribution','nutritionEstimates','yield', "flavours", etc.

        list_ingr=[]
        for ingr_raw in dict_data["ingredientLines"]:
            try:
                list_ingr.append(normalize_ingredient(ingr_raw).name)
            except:
                continue
        # use flavours etc.
        recipes.append({
            "id":i,
            "cuisine":dict_data["attributes"]["cuisine"][0],
            "ingredients":list_ingr,
            "all_ingredients":";".join(list_ingr)    
        })

    df = pd.DataFrame(recipes)
    df.to_pickle(join(PATH,"raw_data.pkl"))
else:
    df=pd.read_pickle(join(PATH,"raw_data.pkl"))
    df=df_train.reset_index()
    NB_RECIPES = len(df)

In [5]:
df['cuisine'].value_counts()

American                11729
Italian                  4571
Mexican                  3394
Asian                    1951
French                   1562
Indian                   1457
Kid-Friendly              962
Southwestern              600
Thai                      347
Barbecue                  338
Chinese                   266
Southern & Soul Food      151
Greek                      80
Mediterranean              72
Spanish                    58
Cuban                      31
Cajun & Creole             18
Moroccan                   11
Japanese                   10
Irish                       9
English                     6
Hawaiian                    5
German                      4
Hungarian                   3
Portuguese                  2
Vietnamese                  1
Name: cuisine, dtype: int64

In [6]:
df["cuisine"]=df["cuisine"].replace({"Vietnamese":"Asian",
                                     "Portuguese":"Spanish_Portuguese",
                                     "Spanish":"Spanish_Portuguese",
                                     "Irish":"English_Irish",
                                    "English":"English_Irish",
                                     "Japanese":"Asian", # more fusion
                                     "Chinese":"Asian", 
                                     "Greek":"Mediterranean",
                                     "Cajun & Creole":"American",
                                     "Southern & Soul Food":"American",
                                     "Thai":"Asian",
                                     "Southwestern":"Mexican"
                                    })

df = df[~df["cuisine"].isin(["Kid-Friendly","Barbecue"])]

In [7]:
df['cuisine'].value_counts()

American              11898
Italian                4571
Mexican                3994
Asian                  2575
French                 1562
Indian                 1457
Mediterranean           152
Spanish_Portuguese       60
Cuban                    31
English_Irish            15
Moroccan                 11
Hawaiian                  5
German                    4
Hungarian                 3
Name: cuisine, dtype: int64

## Removing cuisines with not enough recipes

In [8]:
RECIPE_THRESHOLD=10
cuisine_count= df['cuisine'].value_counts()
to_drop = [cuisine_count[cuisine_count == el].index[0] for el in cuisine_count if el<RECIPE_THRESHOLD]
df=df[~df["cuisine"].isin(to_drop)]

## Normalizing ingredients

In [10]:
df.to_pickle(join(PATH,"full_data.pkl"))